# 実践演習7-2

Scikit learn の SVMで文書分類を行います。

## 準備

必要なライブラリ等を読み込みます。

In [ ]:
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

## データの読み込み

scipyのloadarffはStringの読み込みに対応していないので、直接ファイルを開いて読み込みます。学習データをX, yに読み込みます。

In [ ]:
# 学習データ
r = re.compile("'(.*)',([01])")
X = np.empty((0,1),np.string_)
y = np.empty((0,1),np.string_)

with open('ReutersGrain-train.arff', 'r') as file:
    line = file.readline()
    while not(re.match(r"@data", line)):
        line = file.readline()
    line = file.readline()  # @data のあとの空行を読み飛ばす
    for line in file:
        m = r.search(line)
        X = np.append(X, m.group(1))
        y = np.append(y, m.group(2))

学習データの事例数を確認します。

評価データをXe, yeに読み込みます。

In [ ]:
# 評価データ
Xe = np.empty((0,1),np.string_)
ye = np.empty((0,1),np.string_)
with open('ReutersGrain-test.arff', 'r') as file:
    line = file.readline()
    while not(re.match(r"@data", line)):
        line = file.readline()
    line = file.readline()  # @data のあとの空行を読み飛ばす
    for line in file:
        m = r.search(line)
        Xe = np.append(Xe, m.group(1))
        ye = np.append(ye, m.group(2))

評価データの事例数を確認します。

## 文字列をベクトルに変換

[CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)を使って、文字列を単語ベクトルに変換します。

学習データ中の語彙を次元として、ベクトルに変換します。

評価データは、学習データの次元に合わせて、ベクトルに変換します。

## SVMでの学習

[SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)を使います。学習データの数が少ないので、線形カーネルを用います。また、正例と負例の数が大きく違うので、[class\_weight](http://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane_unbalanced.html#sphx-glr-auto-examples-svm-plot-separating-hyperplane-unbalanced-py)を設定しておきます。

分割学習法の場合は、[classification_report](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)に正解と識別器の出力を与えます。

positiveのF値が0.85となり、Wekaの0.771を上回っています。これば特徴ベクトルの次元数が違うことと、SVMの実装が若干異なることが原因だと思われます。

### 多項式カーネル

2次の多項式カーネルを試してみます。カーネルの定義は[ここ](http://scikit-learn.org/stable/modules/svm.html#svm-kernels)を参照。

### rbfカーネル

rbfカーネルを試してみます。gammaの値を調整することでpositiveの性能が変わります。$\gamma$が小さいと識別面の決定に寄与する学習データが少なくなるので、比較的単純な識別面になります。反対に$\gamma$が大きいと複雑な識別面になります。  

$$K(x, x')=\exp(-\gamma\|x-x'\|^2)$$

### 単語ベクトルの調整

単語数を減らしてみます。

stop wordsを有効にしてみます。

tfidfを使ってみます。